In [158]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keplergl import KeplerGl
import sys
# isplay all columns
pd.set_option('display.max_columns', None)
sys.path.append('../utils/')
from functions import get_zone_info

In [159]:
# Load data
cph = gpd.read_file('../data/processed/cop_area.shp')
dr_trips = pd.read_csv('../data/processed/dr_oneway_spacial.csv')
car_trips = pd.read_csv('../data/processed/alltrips_cars.csv')
# read the txt file "stops.txt" and create a pandas dataframe
stops = pd.read_csv('../data/processed/stops.txt', sep=',')

C:\Users\mihai\AppData\Local\Temp\ipykernel_22896\175736875.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  car_trips = pd.read_csv('../data/processed/alltrips_cars.csv')


In [160]:
car_trips.head()

,vehicleId,PersonID,LatitudeStart,LongitudeStart,LatitudeEnd,LongitudeEnd,StartTime,EndTime,Age,Gender,KmDriven,FromZoneID,ToZoneID,TripDurationHours,TripDurationMinutes,AvgSpeed,Month,Weekday,StartHour,HaversineDistance,StartTimeUpdated,TotalDurationMin,TripDistanceKm,TotalWalkingTimeMin,TotalTransitTimeMin,Changes,PickupStationProximity,DropoffStationProximity,CarDurationMin,CarDistanceKm,source
0,206.0,95281.0,55.67511,12.58435,55.67533,12.58326,2017-07-10 08:43:57,2017-07-10 08:49:17,22.0,M,2.0,102110,102110,0.088889,5.333333,14.173006,July,Monday,8,0.072613,2023-12-25 08:43:57,0.950000,0.07457,0.950000,0.0,0.0,74.57,74.57,0.200000,0.07457,greenmobility
1,30.0,103791.0,55.67554,12.58329,55.67537,12.58320,2017-07-16 08:13:52,2017-07-16 08:35:48,21.0,M,2.0,102110,102110,0.365556,21.933333,3.446324,July,Sunday,8,0.019734,2023-12-24 08:13:52,1.016667,0.07118,1.016667,0.0,0.0,71.18,71.18,0.316667,0.07118,greenmobility
2,17.0,103996.0,55.67901,12.58150,55.67537,12.58327,2017-07-16 11:56:26,2017-07-16 12:03:38,21.0,M,1.0,102120,102110,0.120000,7.200000,4.666562,July,Sunday,11,0.419819,2023-12-24 11:56:26,7.883333,0.53582,7.883333,0.0,0.0,535.82,535.82,3.250000,1.13651,greenmobility
3,144.0,166029.0,55.68043,12.58109,55.67414,12.57805,2017-08-23 14:59:25,2017-08-23 15:28:24,21.0,M,1.0,102120,102110,0.483056,28.983333,1.159261,August,Wednesday,14,0.725149,2023-12-27 14:59:25,13.566667,0.96289,13.566667,0.0,0.0,962.89,962.89,4.266667,1.66995,greenmobility
4,48.0,172263.0,55.67881,12.58385,55.67535,12.58327,2017-08-26 17:58:05,2017-08-26 18:07:59,34.0,M,1.0,102120,102110,0.165000,9.900000,3.393863,August,Saturday,17,0.386571,2023-12-23 17:58:05,8.000000,0.52777,8.000000,0.0,0.0,527.77,527.77,3.150000,0.99055,greenmobility


In [98]:
# keep columns: stop_name, stop_lat, stop_lon
stops = stops[['stop_name', 'stop_lat', 'stop_lon']]
stops.head()

,stop_name,stop_lat,stop_lon
0,Dyrupgårds Alle (Odense Kommune),55.354918,10.345035
1,Vipperød St.,55.666828,11.738710
2,Holbæk St.,55.715750,11.708598
3,Hvalsø St.,55.595537,11.859821
4,Tølløse St.,55.611893,11.772803


In [99]:
dr_trips.columns

Index(['StartTime', 'EndTime', 'UserId', 'latitudeStart', 'longitudeStart',
       'nameStart', 'latitudeEnd', 'longitudeEnd', 'nameEnd',
       'StartHubClusterId', 'EndHubClusterId', 'StartClusterName',
       'EndClusterName', 'StartClusterLatitude', 'StartClusterLongitude',
       'EndClusterLatitude', 'EndClusterLongitude', 'HaversineDistance',
       'Month', 'StartHour', 'Weekday', 'TripDurationMin', 'TripDurationHr',
       'SpeedKmHr', 'RoundTrip', 'Route', 'YearMonth', 'DurationGroup',
       'StartTimeUpdated', 'TotalDurationMin', 'TripDistanceKm',
       'TotalWalkingTimeMin', 'TotalTransitTimeMin', 'Changes',
       'PickupStationProximity', 'DropoffStationProximity', 'SpeedKmHrHav',
       'SpeedKmHrP', 'DistanceType', 'UserCategory', 'UserCluster', 'DayType'],
      dtype='object')

In [100]:
cph.head()

,OBJECTID,komnr,zoneid,zonenavn,zonedescri,numadresse,Shape_Leng,Shape_Area,geometry
0,6585,153,153035,Brøndby Strand NV,Brøndby Kommune,469,5881.304215,6.368232e+05,"MULTIPOLYGON (((717065.090 6167793.410, 717080..."
1,6586,153,153033,Brøndby Strand SV,Brøndby Kommune,567,4178.555401,8.232537e+05,"POLYGON ((714912.062 6169465.885, 714940.877 6..."
2,6588,187,187011,Vallensbæk S,Vallensbæk Kommune,805,7083.711295,1.352975e+06,"POLYGON ((714253.370 6168940.880, 714265.630 6..."
3,6589,153,153021,Brøndbyvester S,Brøndby Kommune,1116,8278.683910,3.304261e+06,"POLYGON ((715846.736 6170548.621, 715502.058 6..."
4,6590,153,153024,Brøndbyvester NNØ,Brøndby Kommune,598,3947.614780,6.585726e+05,"POLYGON ((714678.550 6173491.270, 714675.401 6..."


In [101]:
cph.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    1010 non-null   int64   
 1   komnr       1010 non-null   int64   
 2   zoneid      1010 non-null   int64   
 3   zonenavn    1010 non-null   object  
 4   zonedescri  1010 non-null   object  
 5   numadresse  1010 non-null   int64   
 6   Shape_Leng  1010 non-null   float64 
 7   Shape_Area  1010 non-null   float64 
 8   geometry    1010 non-null   geometry
dtypes: float64(2), geometry(1), int64(4), object(2)
memory usage: 71.1+ KB


In [102]:
# sort cph by zonedescri and print head
cph = cph.sort_values(by='zonedescri')
cph.head()

,OBJECTID,komnr,zoneid,zonenavn,zonedescri,numadresse,Shape_Leng,Shape_Area,geometry
204,6622,165,165032,Albertslund syd S,Albertslund Kommune,719,2833.998761,3.999415e+05,"POLYGON ((710843.199 6173082.418, 710889.352 6..."
913,7366,165,165023,Albertslund syd Station,Albertslund Kommune,284,2864.936954,4.434655e+05,"POLYGON ((711293.725 6173197.729, 710985.236 6..."
914,7367,165,165022,Albertslund syd N,Albertslund Kommune,158,2870.324874,4.518948e+05,"POLYGON ((711081.763 6174210.545, 711103.730 6..."
30,6666,165,165012,Albertslund nord SØ,Albertslund Kommune,1334,4701.005062,1.003677e+06,"POLYGON ((712827.337 6175339.891, 712826.940 6..."
213,7427,165,165015,Albertslund nord NV,Albertslund Kommune,74,13819.909903,5.954921e+06,"POLYGON ((710581.520 6178346.360, 710627.110 6..."


In [103]:
aggregated_gdf = cph.dissolve(by='zonedescri', as_index=False)


In [104]:
aggregated_gdf.head()

,zonedescri,geometry,OBJECTID,komnr,zoneid,zonenavn,numadresse,Shape_Leng,Shape_Area
0,Albertslund Kommune,"POLYGON ((710599.140 6171955.530, 710597.460 6...",6622,165,165032,Albertslund syd S,719,2833.998761,3.999415e+05
1,Allerød Kommune,"POLYGON ((705703.640 6191011.130, 705699.320 6...",6440,201,201024,Allerød CV,540,3950.717975,7.883861e+05
2,Ballerup Kommune,"POLYGON ((711571.540 6178267.240, 711559.640 6...",7755,151,151091,Kildedal,49,8546.735381,2.456872e+06
3,Brøndby Kommune,"MULTIPOLYGON (((714938.550 6168297.450, 714919...",6592,153,153023,Brøndbyvester Midt,558,4045.871436,9.080656e+05
4,Dragør Kommune,"MULTIPOLYGON (((725453.220 6162606.340, 725459...",6569,155,155016,Dragør CS,664,3363.981955,6.992127e+05


In [105]:
#map.add_data(data=aggregated_gdf, name='Aggregated Zones')
#map.add_data(data=cph, name='Copenhagen Area')

In [106]:
#map

In [107]:
# see if there is any invalid geometry in aggregated_gdf
aggregated_gdf.is_valid.unique()

array([ True])

In [108]:
def get_zone_info1(shpfile, clustername, longitudes, latitudes, condition='start'):
    """
    Retrieve zone information based on given points' longitude and latitude.

    Args:
        shpfile_path (str): Path to the shapefile.
        longitudes (iterable): Longitudes of the points.
        latitudes (iterable): Latitudes of the points.

    Returns:
        pandas.DataFrame: A DataFrame containing zone information for each point.
            - zoneID: 6-digit zone ID as in the shapefile.
            - zoneName: Name of the zone as in the shapefile.
            - clustername: Cluster name.
    """

    # convert to WGS84
    new_gdf = shpfile.to_crs("EPSG:4326")

    # Create GeoDataFrame from provided longitudes and latitudes
    points_df = pd.DataFrame({
        'Longitude': longitudes,
        'Latitude': latitudes,
        "ClusterName": clustername,
    })
    points_gdf = gpd.GeoDataFrame(points_df, geometry=gpd.points_from_xy(points_df.Longitude, points_df.Latitude), crs="EPSG:4326")

    # Perform spatial join
    joined_gdf = gpd.sjoin(points_gdf, new_gdf, how="left", predicate='intersects',)

    # Select relevant columns and rename them, zonenavn, zonedescri
    zone_info_df = joined_gdf[['zoneid', 'zonenavn', 'zonedescri', 'ClusterName']].copy()
    if condition == 'start':
        zone_info_df.columns = ['StartZoneId', 'StartZoneName', 'StartZoneDescri', 'StartClusterName']
    else:
        zone_info_df.columns = ['EndZoneId', 'EndZoneName', 'EndZoneDescri', 'EndClusterName']

    return zone_info_df

# Example usage:
# longitudes = [longitude1, longitude2, ...]
# latitudes = [latitude1, latitude2, ...]
# zone_info_df = get_zone_info(shpfile_path, longitudes, latitudes)


In [109]:
import pandas as pd
import geopandas as gpd

def get_zone_info_filtered(shpfile, df, station_col, longitude_col, latitude_col):
    """
    Filter a DataFrame based on whether points fall within zones defined in a shapefile,
    and add zone information to those points.

    Args:
        shpfile (geopandas.GeoDataFrame): GeoDataFrame of the shapefile with zone shapes.
        df (pandas.DataFrame): DataFrame with points to check, including longitude and latitude.
        clustername_col (str): Column name in df for cluster names.
        longitude_col (str): Column name in df for longitudes.
        latitude_col (str): Column name in df for latitudes.

    Returns:
        pandas.DataFrame: Original DataFrame with added 'ZoneId' and 'ZoneName' where applicable.
    """
    
    # Convert shapefile and points to WGS84 if not already
    shpfile = shpfile.to_crs("EPSG:4326")
    station_col = df[station_col].name
    # Create GeoDataFrame from DataFrame's longitude and latitude
    points_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[longitude_col], df[latitude_col]), crs="EPSG:4326")

    # Perform spatial join to find which points fall within which zones
    joined_gdf = gpd.sjoin(points_gdf, shpfile, how="left", predicate='intersects')
    
    # Rename columns for zone ID and name from shapefile, assuming they are named 'zoneid' and 'zonenavn' in shpfile
    joined_gdf = joined_gdf[['zoneid', 'zonenavn', station_col, latitude_col, longitude_col,]]
    #joined_gdf.rename(columns={'zoneid': 'ZoneId', 'zonenavn': 'ZoneName'}, inplace=True)

    
    # Add cluster name column from specified clustername_col, if not already present
    return joined_gdf

In [110]:
stops['stop_name'].name

'stop_name'

In [111]:
filtered_stops = get_zone_info_filtered(cph, stops, 'stop_name', 'stop_lon', 'stop_lat')


In [112]:
filtered_stops.head()

,zoneid,zonenavn,stop_name,stop_lat,stop_lon
0,NaN,NaN,Dyrupgårds Alle (Odense Kommune),55.354918,10.345035
1,NaN,NaN,Vipperød St.,55.666828,11.738710
2,NaN,NaN,Holbæk St.,55.715750,11.708598
3,350033.0,Nørre Hvalsø,Hvalsø St.,55.595537,11.859821
4,NaN,NaN,Tølløse St.,55.611893,11.772803


In [113]:
# drop nan values
filtered_stops = filtered_stops.dropna()

In [114]:
filtered_stops.head()

,zoneid,zonenavn,stop_name,stop_lat,stop_lon
3,350033.0,Nørre Hvalsø,Hvalsø St.,55.595537,11.859821
5,250033.0,Frederikssund CS,Frederikssund St.,55.835809,12.065636
6,350021.0,Lejre,Lejre St.,55.604804,11.971702
7,240063.0,Ølstykke SØ,Ølstykke St.,55.795641,12.159712
9,259057.0,Køge Nord SV,Ølby St.,55.479751,12.175762


In [115]:
cph.is_valid.value_counts()

True     1008
False       2
Name: count, dtype: int64

In [116]:
# fix invalid geometries in geometry column
cph['geometry'] = cph.geometry.buffer(0)

In [117]:
zoneinfo_dataframe = get_zone_info1(cph,dr_trips["EndClusterName"], dr_trips['longitudeEnd'].values, 
                                    dr_trips['latitudeEnd'].values, condition = 'end')

In [118]:
# print head of rows where EndzoneID is null 
zoneinfo_dataframe[zoneinfo_dataframe['EndZoneId'].isnull()].head()

,EndZoneId,EndZoneName,EndZoneDescri,EndClusterName
1349,NaN,NaN,NaN,Havnegade Harbour Promenade
2146,NaN,NaN,NaN,Havnegade Harbour Promenade
2500,NaN,NaN,NaN,Havnegade Harbour Promenade
4072,NaN,NaN,NaN,Havnegade Harbour Promenade
5961,NaN,NaN,NaN,Havnegade Harbour Promenade


In [119]:
# print nr of nan values
zoneinfo_dataframe.isna().sum()

EndZoneId         168
EndZoneName       168
EndZoneDescri     168
EndClusterName      0
dtype: int64

In [120]:
zoneinfo_dataframe.iloc[:,:3]

,EndZoneId,EndZoneName,EndZoneDescri
0,103173.0,Sundbyvester NNØ,"Københavns Kommune, Amager"
1,102821.0,Vesterbro N,Københavns Kommune syd for Frederiksberg
2,103193.0,Islands Brygge S,"Københavns Kommune, Amager"
3,103193.0,Islands Brygge S,"Københavns Kommune, Amager"
4,102541.0,Utterslev NØ,Københavns Kommune Nord
...,...,...,...
82247,102442.0,Indre Nørrebro Ø,Københavns Kommune Nord
82248,102131.0,Rådhuspladsen,"Københavns Kommune, indre by og Christianshavn"
82249,147233.0,Frederiksberg Vest Midt NØ,Frederiksberg Kommune
82250,102141.0,Rundetårn,"Københavns Kommune, indre by og Christianshavn"


In [121]:
start_zoneinfo_dataframe = get_zone_info1(cph, dr_trips["StartClusterName"], dr_trips['longitudeStart'].values, dr_trips['latitudeStart'].values, condition = 'start')

In [122]:
# append first 3 columns of zoneinfo_dataframe to dr_trips
dr_trips = pd.concat([dr_trips, start_zoneinfo_dataframe.iloc[:,:3]], axis=1)
dr_trips = pd.concat([dr_trips, zoneinfo_dataframe.iloc[:,:3]], axis=1)

In [123]:
dr_trips.head()

,StartTime,EndTime,UserId,latitudeStart,longitudeStart,nameStart,latitudeEnd,longitudeEnd,nameEnd,StartHubClusterId,EndHubClusterId,StartClusterName,EndClusterName,StartClusterLatitude,StartClusterLongitude,EndClusterLatitude,EndClusterLongitude,HaversineDistance,Month,StartHour,Weekday,TripDurationMin,TripDurationHr,SpeedKmHr,RoundTrip,Route,YearMonth,DurationGroup,StartTimeUpdated,TotalDurationMin,TripDistanceKm,TotalWalkingTimeMin,TotalTransitTimeMin,Changes,PickupStationProximity,DropoffStationProximity,SpeedKmHrHav,SpeedKmHrP,DistanceType,UserCategory,UserCluster,DayType,StartZoneId,StartZoneName,StartZoneDescri,EndZoneId,EndZoneName,EndZoneDescri
0,2018-03-01 17:43:14.707445,2018-03-01 18:14:12.145000,108186,55.673440,12.564409,Central Station,55.658239,12.605434,Skotlands Plads,590,331,Central Station,Skotlands Plads,55.673117,12.564212,55.658397,12.605787,3.078446,March,17,Thursday,30.957293,0.515955,6.008277,False,Central Station -> Skotlands Plads,2018-03,0-1 hour,2023-11-16 17:43:14.707445,17.500000,3.86560,6.500000,11.0,0.0,324.78,125.35,5.966503,7.492128,Medium,Occasional Users,1,Weekday,102182.0,Københavns Hovedbanegård,Københavns Kommune syd for Frederiksberg,103173.0,Sundbyvester NNØ,"Københavns Kommune, Amager"
1,2018-03-03 13:22:27.557683,2018-03-03 13:40:09.817000,109897,55.673668,12.571551,H. C. Andersens Blvd.,55.668555,12.547020,Broagergade,579,564,Ved Glyptoteket,Broagergade,55.672956,12.570784,55.669073,12.546464,1.640003,March,13,Saturday,17.704322,0.295072,10.505909,False,Ved Glyptoteket -> Broagergade,2018-03,0-1 hour,2023-11-18 13:22:27.557683,15.533333,2.69815,10.533333,5.0,0.0,80.07,677.63,5.557974,9.144038,Short,Occasional Users,2,Weekend,102172.0,Tivoli,"Københavns Kommune, indre by og Christianshavn",102821.0,Vesterbro N,Københavns Kommune syd for Frederiksberg
2,2018-03-06 07:45:32.192047,2018-03-06 08:09:27.571000,1,55.672142,12.590571,Christianshavn,55.665808,12.578714,Egilsgade,50,542,Christianshavn,Egilsgade,55.672121,12.590597,55.665262,12.579198,1.024221,March,7,Tuesday,23.922983,0.398716,7.774950,False,Christianshavn -> Egilsgade,2018-03,0-1 hour,2023-11-21 07:45:32.192047,15.233333,1.87281,14.233333,1.0,0.0,100.86,538.09,2.568796,4.697098,Short,Moderately Active Users,2,Weekday,103142.0,Christianshavn S,"Københavns Kommune, indre by og Christianshavn",103193.0,Islands Brygge S,"Københavns Kommune, Amager"
3,2018-03-07 09:14:43.451300,2018-03-07 09:44:31.006000,14,55.682597,12.571227,Nørreport St.,55.666573,12.580692,Founders House,40,538,Nørreport St.,Founders House,55.683061,12.570524,55.666721,12.579745,1.878027,March,9,Wednesday,29.792578,0.496543,6.243166,False,Nørreport St. -> Founders House,2018-03,0-1 hour,2023-11-15 09:14:43.451300,22.550000,4.49931,12.333333,8.0,1.0,190.22,116.28,3.782205,9.061270,Short,Occasional Users,2,Weekday,102141.0,Rundetårn,"Københavns Kommune, indre by og Christianshavn",103193.0,Islands Brygge S,"Københavns Kommune, Amager"
4,2018-03-08 08:57:15.603150,2018-03-08 09:33:33.622000,114469,55.668653,12.543646,Vega,55.701914,12.533708,Frederikssundsvej,562,267,Vega,Frederikssundsvej,55.668626,12.543944,55.702002,12.533122,3.750543,March,8,Thursday,36.300314,0.605005,5.123923,False,Vega -> Frederikssundsvej,2018-03,0-1 hour,2023-11-16 08:57:15.603150,23.733333,5.81268,13.733333,10.0,0.0,267.99,375.18,6.199191,9.607652,Medium,Occasional Users,1,Weekday,102821.0,Vesterbro N,Københavns Kommune syd for Frederiksberg,102541.0,Utterslev NØ,Københavns Kommune Nord


In [124]:
# show rows ehre StartZoneId is null and EndZoneId is null
dr_trips[(dr_trips['StartZoneId'].isnull())]['StartClusterName'].value_counts().index
# for each of the clusters obtained above, find out the StartClusterName which is closest to the location of the cluster
# and assign the StartZoneId, StartZoneName, and StartZoneDescri to the rows where StartZoneId is null 


Index(['Havnegade Harbour Promenade', 'Nordhavn St.', 'Corkgade'], dtype='object', name='StartClusterName')

In [125]:
import pandas as pd
from haversine import haversine

# Assuming dr_trips is your DataFrame
# Sample structure for dr_trips (replace with your actual DataFrame setup)

# Function to calculate Haversine distance
def calculate_haversine(lat1, lon1, lat2, lon2):
    """Calculate the Haversine distance between two points in kilometers."""
    return haversine((lat1, lon1), (lat2, lon2), unit='km')

# Identify clusters with missing StartZoneId and prepare data for all clusters
missing_clusters = dr_trips[dr_trips['StartZoneId'].isnull()]['StartClusterName'].unique()
all_clusters = dr_trips[['StartClusterName', 'StartClusterLatitude', 'StartClusterLongitude']].drop_duplicates()

# Iterate over missing clusters to calculate distances and perform imputation
for missing_cluster in missing_clusters:
    missing_lat = dr_trips.loc[dr_trips['StartClusterName'] == missing_cluster, 'StartClusterLatitude'].iloc[0]
    missing_lon = dr_trips.loc[dr_trips['StartClusterName'] == missing_cluster, 'StartClusterLongitude'].iloc[0]

    # Calculate distances from the missing cluster to all other clusters
    distances = []
    for _, row in all_clusters.iterrows():
        if row['StartClusterName'] != missing_cluster:
            distance = calculate_haversine(missing_lat, missing_lon, row['StartClusterLatitude'], row['StartClusterLongitude'])
            distances.append((row['StartClusterName'], distance))

    # Sort distances and take the closest cluster
    closest_cluster_name = sorted(distances, key=lambda x: x[1])[0][0]

    # Find the row index(es) of the closest cluster with valid StartZoneId for imputation
    closest_cluster_info = dr_trips[dr_trips['StartClusterName'] == closest_cluster_name].iloc[0]

    # Impute the missing values
    dr_trips.loc[dr_trips['StartClusterName'] == missing_cluster, 'StartZoneId'] = closest_cluster_info['StartZoneId']
    dr_trips.loc[dr_trips['StartClusterName'] == missing_cluster, 'StartZoneName'] = closest_cluster_info['StartZoneName']
    dr_trips.loc[dr_trips['StartClusterName'] == missing_cluster, 'StartZoneDescri'] = closest_cluster_info['StartZoneDescri']
    dr_trips.loc[dr_trips['EndClusterName'] == missing_cluster, 'EndZoneId'] = closest_cluster_info['EndZoneId']
    dr_trips.loc[dr_trips['EndClusterName'] == missing_cluster, 'EndZoneName'] = closest_cluster_info['EndZoneName']
    dr_trips.loc[dr_trips['EndClusterName'] == missing_cluster, 'EndZoneDescri'] = closest_cluster_info['EndZoneDescri']

    # Optional: Print the closest clusters for verification
    print(f"Missing Cluster: {missing_cluster}, Closest Cluster: {closest_cluster_name}, Distance: {sorted(distances, key=lambda x: x[1])[0][1]} km")


Missing Cluster: Havnegade Harbour Promenade, Closest Cluster: Havnegade, Distance: 0.18626914819973797 km
Missing Cluster: Nordhavn St., Closest Cluster: Hjørringgade, Distance: 0.1670189887317229 km
Missing Cluster: Corkgade, Closest Cluster: Hamborg Plads, Distance: 0.21973679349776398 km


In [126]:
dr_trips[(dr_trips['EndZoneId'].isnull())]['EndClusterName'].value_counts()

Series([], Name: count, dtype: int64)

In [127]:
dr_trips.columns    

Index(['StartTime', 'EndTime', 'UserId', 'latitudeStart', 'longitudeStart',
       'nameStart', 'latitudeEnd', 'longitudeEnd', 'nameEnd',
       'StartHubClusterId', 'EndHubClusterId', 'StartClusterName',
       'EndClusterName', 'StartClusterLatitude', 'StartClusterLongitude',
       'EndClusterLatitude', 'EndClusterLongitude', 'HaversineDistance',
       'Month', 'StartHour', 'Weekday', 'TripDurationMin', 'TripDurationHr',
       'SpeedKmHr', 'RoundTrip', 'Route', 'YearMonth', 'DurationGroup',
       'StartTimeUpdated', 'TotalDurationMin', 'TripDistanceKm',
       'TotalWalkingTimeMin', 'TotalTransitTimeMin', 'Changes',
       'PickupStationProximity', 'DropoffStationProximity', 'SpeedKmHrHav',
       'SpeedKmHrP', 'DistanceType', 'UserCategory', 'UserCluster', 'DayType',
       'StartZoneId', 'StartZoneName', 'StartZoneDescri', 'EndZoneId',
       'EndZoneName', 'EndZoneDescri'],
      dtype='object')

In [128]:
# rename the occurences where Bermuda Triangle is the StartClusterName or EndClusterName to Kongens Nytorv
dr_trips['StartClusterName'] = dr_trips['StartClusterName'].replace('Bermuda Triangle', 'Kongens Nytorv')
dr_trips['EndClusterName'] = dr_trips['EndClusterName'].replace('Bermuda Triangle', 'Kongens Nytorv')


In [129]:
common_values = np.intersect1d(dr_trips['StartZoneDescri'].unique(), dr_trips['EndZoneDescri'].unique())
common_values

# filter aggregated_gdf to only include the common values
aggregated_gdf = aggregated_gdf[aggregated_gdf['zonedescri'].isin(common_values)]


In [130]:
common_values = np.intersect1d(dr_trips['StartZoneId'].unique(), dr_trips['EndZoneId'].unique())
common_values

# filter aggregated_gdf to only include the common values
cph = cph[cph['zoneid'].isin(common_values)]

In [131]:
# group by StartZoneId and count the number of trips
start_zone_counts = dr_trips.groupby('StartZoneId').size().reset_index(name='StartZoneCounts')
# merge with dr_trips on StartZoneId
dr_trips = dr_trips.merge(start_zone_counts, on='StartZoneId', how='left')
# rename startzoneid to zoneid
start_zone_counts = start_zone_counts.rename(columns={'StartZoneId': 'zoneid'})
# group by  EndZoneId and count the number of trips
end_zone_counts = dr_trips.groupby('EndZoneId').size().reset_index(name='EndZoneCounts')
# merge with dr_trips on EndZoneId
dr_trips = dr_trips.merge(end_zone_counts, on='EndZoneId', how='left')
# rename endzoneid to zoneid
end_zone_counts = end_zone_counts.rename(columns={'EndZoneId': 'zoneid'})
# create a column called TripDurationDiff where we subtract TripDurationMin from TotalDurationMin
dr_trips['TripDurationDiff'] = dr_trips['TotalDurationMin'] - dr_trips['TripDurationMin']
# group StartZoneName and get the mean of TripDurationDiff
start_zone_duration = dr_trips.groupby('StartZoneName')['TripDurationDiff'].mean().reset_index(name='BikePubTranspTimeDiff')
# rename startzonename to zonenavn
start_zone_duration = start_zone_duration.rename(columns={'StartZoneName': 'zonenavn'})

# merge with cph
cph = cph.merge(start_zone_counts, on='zoneid', how='left')
cph = cph.merge(end_zone_counts, on='zoneid', how='left')
cph = cph.merge(start_zone_duration, on='zonenavn', how='left')


In [132]:
dr_trips[['TotalDurationMin','TripDurationMin']].describe()

,TotalDurationMin,TripDurationMin
count,82252.000000,82252.000000
mean,22.396652,23.173378
std,7.953310,13.995551
min,1.850000,1.042302
25%,16.783333,14.647366
50%,21.850000,20.624985
75%,27.366667,27.661646
max,72.016667,119.918954


In [157]:
# count nr of rows where TripDurationMin > 60 
dr_trips[dr_trips['TripDurationMin'] > 60].shape[0]

2231

In [133]:
dr_trip_no_outliers = dr_trips[(dr_trips['TripDurationMin'] <= 60)]


In [134]:
dr_trip_no_outliers['TripDurationDiff'].describe()

count    80021.000000
mean         0.616796
std          8.919976
min        -48.895513
25%         -4.011953
50%          1.646172
75%          6.441792
max         37.177437
Name: TripDurationDiff, dtype: float64

In [135]:
start_zone_duration.describe()

,BikePubTranspTimeDiff
count,221.000000
mean,-3.225875
std,7.325982
min,-39.213477
25%,-4.293993
50%,-1.331598
75%,0.848721
max,15.090793


In [136]:
# group StartZoneName and get the mean of TripDurationDiff
start_zone_duration_nooutlier = dr_trip_no_outliers.groupby('StartZoneName')['TripDurationDiff'].mean().reset_index(name='TimeDiffTrimmed')
start_zone_duration_nooutlier = start_zone_duration_nooutlier.rename(columns={'StartZoneName': 'zonenavn'})
# merge with cph
cph = cph.merge(start_zone_duration_nooutlier, on='zonenavn', how='left')

In [139]:
cph.head()

,OBJECTID,komnr,zoneid,zonenavn,zonedescri,numadresse,Shape_Leng,Shape_Area,geometry,StartZoneCounts,EndZoneCounts,BikePubTranspTimeDiff,TimeDiffTrimmed
0,7563,153,153013,Brøndbyøster Station,Brøndby Kommune,294,2061.521377,231914.742604,"POLYGON ((716663.080 6173680.586, 716530.495 6...",4,12,-19.648724,-6.959153
1,7561,153,153022,Brøndbyvester Ø,Brøndby Kommune,37,4183.953600,823963.794578,"POLYGON ((715425.332 6172855.164, 715442.975 6...",10,17,2.682670,11.794615
2,7741,147,147132,Frederiksberg Øst S NV,Frederiksberg Kommune,465,2591.563566,409535.151366,"POLYGON ((722267.301 6175896.238, 722495.718 6...",436,468,-2.658458,-1.403027
3,7383,147,147151,Frederiksberg Øst V Ø,Frederiksberg Kommune,109,1365.948589,111873.900827,"POLYGON ((722141.163 6175936.834, 722140.054 6...",26,31,0.034742,0.034742
4,7750,147,147252,Frederiksberg Vest V N,Frederiksberg Kommune,394,2761.685385,374065.077650,"POLYGON ((719994.072 6176484.480, 720136.777 6...",17,13,-2.064785,-2.064785


In [140]:
# see the descrition of HaversineDistance for TripDurationMin >= 60
dr_trips[dr_trips['TripDurationMin'] >= 60]['HaversineDistance'].describe()

count    2231.000000
mean        5.074293
std         1.751166
min         2.509934
25%         3.862739
50%         4.677574
75%         5.772590
max        14.453685
Name: HaversineDistance, dtype: float64

In [141]:
cph.describe()

,OBJECTID,komnr,zoneid,numadresse,Shape_Leng,Shape_Area,StartZoneCounts,EndZoneCounts,BikePubTranspTimeDiff,TimeDiffTrimmed
count,219.000000,219.000000,219.000000,219.000000,219.000000,2.190000e+02,219.000000,219.000000,219.000000,219.000000
mean,7156.949772,118.552511,119791.684932,371.470320,3325.324376,5.729596e+05,375.557078,375.561644,-3.313016,-0.537836
std,1380.927588,28.952995,28287.897218,239.724695,4021.269263,1.243501e+06,531.518743,499.399134,7.301078,4.907512
min,5659.000000,101.000000,102111.000000,0.000000,989.147517,5.492794e+04,1.000000,1.000000,-39.213477,-22.852790
25%,6029.000000,101.000000,102442.500000,202.500000,2120.023587,2.295557e+05,25.000000,26.500000,-4.362581,-1.800737
50%,7344.000000,101.000000,102863.000000,328.000000,2745.596429,3.737339e+05,159.000000,171.000000,-1.355360,0.307537
75%,7527.000000,147.000000,147132.500000,533.500000,3543.551727,5.528697e+05,469.500000,479.500000,0.827647,2.052864
max,11669.000000,185.000000,185204.000000,1211.000000,58392.876472,1.703654e+07,2768.000000,2237.000000,15.090793,15.090793


In [142]:
# filter filtered_stops to have only the rows where zoneid is in cph
filtered_stops = filtered_stops[filtered_stops['zoneid'].isin(cph['zoneid'])]

In [143]:
dr_trips.head()

,StartTime,EndTime,UserId,latitudeStart,longitudeStart,nameStart,latitudeEnd,longitudeEnd,nameEnd,StartHubClusterId,EndHubClusterId,StartClusterName,EndClusterName,StartClusterLatitude,StartClusterLongitude,EndClusterLatitude,EndClusterLongitude,HaversineDistance,Month,StartHour,Weekday,TripDurationMin,TripDurationHr,SpeedKmHr,RoundTrip,Route,YearMonth,DurationGroup,StartTimeUpdated,TotalDurationMin,TripDistanceKm,TotalWalkingTimeMin,TotalTransitTimeMin,Changes,PickupStationProximity,DropoffStationProximity,SpeedKmHrHav,SpeedKmHrP,DistanceType,UserCategory,UserCluster,DayType,StartZoneId,StartZoneName,StartZoneDescri,EndZoneId,EndZoneName,EndZoneDescri,StartZoneCounts,EndZoneCounts,TripDurationDiff
0,2018-03-01 17:43:14.707445,2018-03-01 18:14:12.145000,108186,55.673440,12.564409,Central Station,55.658239,12.605434,Skotlands Plads,590,331,Central Station,Skotlands Plads,55.673117,12.564212,55.658397,12.605787,3.078446,March,17,Thursday,30.957293,0.515955,6.008277,False,Central Station -> Skotlands Plads,2018-03,0-1 hour,2023-11-16 17:43:14.707445,17.500000,3.86560,6.500000,11.0,0.0,324.78,125.35,5.966503,7.492128,Medium,Occasional Users,1,Weekday,102182.0,Københavns Hovedbanegård,Københavns Kommune syd for Frederiksberg,103173.0,Sundbyvester NNØ,"Københavns Kommune, Amager",1632,207,-13.457293
1,2018-03-03 13:22:27.557683,2018-03-03 13:40:09.817000,109897,55.673668,12.571551,H. C. Andersens Blvd.,55.668555,12.547020,Broagergade,579,564,Ved Glyptoteket,Broagergade,55.672956,12.570784,55.669073,12.546464,1.640003,March,13,Saturday,17.704322,0.295072,10.505909,False,Ved Glyptoteket -> Broagergade,2018-03,0-1 hour,2023-11-18 13:22:27.557683,15.533333,2.69815,10.533333,5.0,0.0,80.07,677.63,5.557974,9.144038,Short,Occasional Users,2,Weekend,102172.0,Tivoli,"Københavns Kommune, indre by og Christianshavn",102821.0,Vesterbro N,Københavns Kommune syd for Frederiksberg,797,1834,-2.170989
2,2018-03-06 07:45:32.192047,2018-03-06 08:09:27.571000,1,55.672142,12.590571,Christianshavn,55.665808,12.578714,Egilsgade,50,542,Christianshavn,Egilsgade,55.672121,12.590597,55.665262,12.579198,1.024221,March,7,Tuesday,23.922983,0.398716,7.774950,False,Christianshavn -> Egilsgade,2018-03,0-1 hour,2023-11-21 07:45:32.192047,15.233333,1.87281,14.233333,1.0,0.0,100.86,538.09,2.568796,4.697098,Short,Moderately Active Users,2,Weekday,103142.0,Christianshavn S,"Københavns Kommune, indre by og Christianshavn",103193.0,Islands Brygge S,"Københavns Kommune, Amager",837,1417,-8.689649
3,2018-03-07 09:14:43.451300,2018-03-07 09:44:31.006000,14,55.682597,12.571227,Nørreport St.,55.666573,12.580692,Founders House,40,538,Nørreport St.,Founders House,55.683061,12.570524,55.666721,12.579745,1.878027,March,9,Wednesday,29.792578,0.496543,6.243166,False,Nørreport St. -> Founders House,2018-03,0-1 hour,2023-11-15 09:14:43.451300,22.550000,4.49931,12.333333,8.0,1.0,190.22,116.28,3.782205,9.061270,Short,Occasional Users,2,Weekday,102141.0,Rundetårn,"Københavns Kommune, indre by og Christianshavn",103193.0,Islands Brygge S,"Københavns Kommune, Amager",2768,1417,-7.242578
4,2018-03-08 08:57:15.603150,2018-03-08 09:33:33.622000,114469,55.668653,12.543646,Vega,55.701914,12.533708,Frederikssundsvej,562,267,Vega,Frederikssundsvej,55.668626,12.543944,55.702002,12.533122,3.750543,March,8,Thursday,36.300314,0.605005,5.123923,False,Vega -> Frederikssundsvej,2018-03,0-1 hour,2023-11-16 08:57:15.603150,23.733333,5.81268,13.733333,10.0,0.0,267.99,375.18,6.199191,9.607652,Medium,Occasional Users,1,Weekday,102821.0,Vesterbro N,Københavns Kommune syd for Frederiksberg,102541.0,Utterslev NØ,Københavns Kommune Nord,1850,683,-12.566981


In [144]:
dr_trips.columns

Index(['StartTime', 'EndTime', 'UserId', 'latitudeStart', 'longitudeStart',
       'nameStart', 'latitudeEnd', 'longitudeEnd', 'nameEnd',
       'StartHubClusterId', 'EndHubClusterId', 'StartClusterName',
       'EndClusterName', 'StartClusterLatitude', 'StartClusterLongitude',
       'EndClusterLatitude', 'EndClusterLongitude', 'HaversineDistance',
       'Month', 'StartHour', 'Weekday', 'TripDurationMin', 'TripDurationHr',
       'SpeedKmHr', 'RoundTrip', 'Route', 'YearMonth', 'DurationGroup',
       'StartTimeUpdated', 'TotalDurationMin', 'TripDistanceKm',
       'TotalWalkingTimeMin', 'TotalTransitTimeMin', 'Changes',
       'PickupStationProximity', 'DropoffStationProximity', 'SpeedKmHrHav',
       'SpeedKmHrP', 'DistanceType', 'UserCategory', 'UserCluster', 'DayType',
       'StartZoneId', 'StartZoneName', 'StartZoneDescri', 'EndZoneId',
       'EndZoneName', 'EndZoneDescri', 'StartZoneCounts', 'EndZoneCounts',
       'TripDurationDiff'],
      dtype='object')

In [145]:
# Convert Start and End Times to Unix timestamps (assuming they're in a recognizable datetime format)
dr_trips['StartTime'] = pd.to_datetime(dr_trips['StartTime']).astype('int64') / 10**9
dr_trips['EndTime'] = pd.to_datetime(dr_trips['EndTime']).astype('int64') / 10**9

# Function to create GeoJSON LineString with timestamps
def create_geojson_feature(row):
    # Assuming altitude is 0 for all points, adjust if you have altitude information
    coordinates = [
        [row['longitudeStart'], row['latitudeStart'], 0, row['StartTime']],
        [row['longitudeEnd'], row['latitudeEnd'], 0, row['EndTime']]
    ]
    feature = {
        "type": "Feature",
        "properties": {},  # Add any properties you want to include
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        }
    }
    return feature

# Apply function to create GeoJSON column
dr_trips['GeoJSON'] = dr_trips.apply(create_geojson_feature, axis=1)

# Optionally, convert the individual GeoJSON features into a FeatureCollection
geojson_feature_collection = {
    "type": "FeatureCollection",
    "features": dr_trips['GeoJSON'].tolist()
}

In [146]:
# convert StartZoneId and EndZoneId to categorical
dr_trips['StartZoneId'] = dr_trips['StartZoneId'].astype('category')
dr_trips['EndZoneId'] = dr_trips['EndZoneId'].astype('category')


In [147]:
# drop duplicates
filtered_stops.drop_duplicates(subset=['stop_name'], inplace=True)
# create new categorical column called station_type
# if stop_name contains 'Metro' then station_type is 'Metro'
# if stop name ends with 'St.' then station_type is 'S-Train'
# else station_type is 'Bus'
filtered_stops['station_type'] = np.where(filtered_stops['stop_name'].str.contains('Metro'), 'Metro', 
                                          np.where(filtered_stops['stop_name'].str.endswith('St.'), 'S-Train', 'Bus'))
# set station_type to categorical
filtered_stops['station_type'] = filtered_stops['station_type'].astype('category')

In [148]:
map = KeplerGl( height=700,)
#map.add_data(data=dr_trips, name='Donkey Republic Trips')
map.add_data(data=dr_trips, name='BIke Trips')
map.add_data(data=cph, name='Copenhagen Area')
map.add_data(filtered_stops, name='Public Transport Stops')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [149]:
map

KeplerGl(data={'BIke Trips': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

In [150]:
# save map config settings


In [151]:
map.save_to_html(file_name='kepler_map.html')

Map saved to kepler_map.html!


In [152]:
# Example setup for aggregated_gdf
# aggregated_gdf = gpd.read_file('path_to_your_aggregated_shapefile.shp')  # Adjust as necessary

# Convert geometries to GeoJSON strings
#aggregated_gdf['geometry_geojson'] = aggregated_gdf['geometry'].apply(lambda x: gpd.GeoSeries([x]).to_json())

# Assuming 'zonedescri' is the column in aggregated_gdf that matches with StartZoneDescri and EndZoneDescri in dr_trips
# start_geom_map = aggregated_gdf.set_index('zonedescri')['geometry']
# end_geom_map = aggregated_gdf.set_index('zonedescri')['geometry']
# # Map the GeoJSON strings to dr_trips
# dr_trips['StartGeometry'] = dr_trips['StartZoneDescri'].map(start_geom_map)
# dr_trips['EndGeometry'] = dr_trips['EndZoneDescri'].map(end_geom_map)



In [153]:
dr_trips.head()

,StartTime,EndTime,UserId,latitudeStart,longitudeStart,nameStart,latitudeEnd,longitudeEnd,nameEnd,StartHubClusterId,EndHubClusterId,StartClusterName,EndClusterName,StartClusterLatitude,StartClusterLongitude,EndClusterLatitude,EndClusterLongitude,HaversineDistance,Month,StartHour,Weekday,TripDurationMin,TripDurationHr,SpeedKmHr,RoundTrip,Route,YearMonth,DurationGroup,StartTimeUpdated,TotalDurationMin,TripDistanceKm,TotalWalkingTimeMin,TotalTransitTimeMin,Changes,PickupStationProximity,DropoffStationProximity,SpeedKmHrHav,SpeedKmHrP,DistanceType,UserCategory,UserCluster,DayType,StartZoneId,StartZoneName,StartZoneDescri,EndZoneId,EndZoneName,EndZoneDescri,StartZoneCounts,EndZoneCounts,TripDurationDiff,GeoJSON
0,1.519926e+09,1.519928e+09,108186,55.673440,12.564409,Central Station,55.658239,12.605434,Skotlands Plads,590,331,Central Station,Skotlands Plads,55.673117,12.564212,55.658397,12.605787,3.078446,March,17,Thursday,30.957293,0.515955,6.008277,False,Central Station -> Skotlands Plads,2018-03,0-1 hour,2023-11-16 17:43:14.707445,17.500000,3.86560,6.500000,11.0,0.0,324.78,125.35,5.966503,7.492128,Medium,Occasional Users,1,Weekday,102182.0,Københavns Hovedbanegård,Københavns Kommune syd for Frederiksberg,103173.0,Sundbyvester NNØ,"Københavns Kommune, Amager",1632,207,-13.457293,"{'type': 'Feature', 'properties': {}, 'geometr..."
1,1.520083e+09,1.520084e+09,109897,55.673668,12.571551,H. C. Andersens Blvd.,55.668555,12.547020,Broagergade,579,564,Ved Glyptoteket,Broagergade,55.672956,12.570784,55.669073,12.546464,1.640003,March,13,Saturday,17.704322,0.295072,10.505909,False,Ved Glyptoteket -> Broagergade,2018-03,0-1 hour,2023-11-18 13:22:27.557683,15.533333,2.69815,10.533333,5.0,0.0,80.07,677.63,5.557974,9.144038,Short,Occasional Users,2,Weekend,102172.0,Tivoli,"Københavns Kommune, indre by og Christianshavn",102821.0,Vesterbro N,Københavns Kommune syd for Frederiksberg,797,1834,-2.170989,"{'type': 'Feature', 'properties': {}, 'geometr..."
2,1.520322e+09,1.520324e+09,1,55.672142,12.590571,Christianshavn,55.665808,12.578714,Egilsgade,50,542,Christianshavn,Egilsgade,55.672121,12.590597,55.665262,12.579198,1.024221,March,7,Tuesday,23.922983,0.398716,7.774950,False,Christianshavn -> Egilsgade,2018-03,0-1 hour,2023-11-21 07:45:32.192047,15.233333,1.87281,14.233333,1.0,0.0,100.86,538.09,2.568796,4.697098,Short,Moderately Active Users,2,Weekday,103142.0,Christianshavn S,"Københavns Kommune, indre by og Christianshavn",103193.0,Islands Brygge S,"Københavns Kommune, Amager",837,1417,-8.689649,"{'type': 'Feature', 'properties': {}, 'geometr..."
3,1.520414e+09,1.520416e+09,14,55.682597,12.571227,Nørreport St.,55.666573,12.580692,Founders House,40,538,Nørreport St.,Founders House,55.683061,12.570524,55.666721,12.579745,1.878027,March,9,Wednesday,29.792578,0.496543,6.243166,False,Nørreport St. -> Founders House,2018-03,0-1 hour,2023-11-15 09:14:43.451300,22.550000,4.49931,12.333333,8.0,1.0,190.22,116.28,3.782205,9.061270,Short,Occasional Users,2,Weekday,102141.0,Rundetårn,"Københavns Kommune, indre by og Christianshavn",103193.0,Islands Brygge S,"Københavns Kommune, Amager",2768,1417,-7.242578,"{'type': 'Feature', 'properties': {}, 'geometr..."
4,1.520499e+09,1.520502e+09,114469,55.668653,12.543646,Vega,55.701914,12.533708,Frederikssundsvej,562,267,Vega,Frederikssundsvej,55.668626,12.543944,55.702002,12.533122,3.750543,March,8,Thursday,36.300314,0.605005,5.123923,False,Vega -> Frederikssundsvej,2018-03,0-1 hour,2023-11-16 08:57:15.603150,23.733333,5.81268,13.733333,10.0,0.0,267.99,375.18,6.199191,9.607652,Medium,Occasional Users,1,Weekday,102821.0,Vesterbro N,Københavns Kommune syd for Frederiksberg,102541.0,Utterslev NØ,Københavns Kommune Nord,1850,683,-12.566981,"{'type': 'Feature', 'properties': {}, 'geometr..."


In [154]:
# dr_trips = dr_trips.rename(columns={'StartGeometry': 'geometry'})
# # convert to geodataframe
# dr_trips = gpd.GeoDataFrame(dr_trips, geometry='geometry')
# # rename StartGeometry to geometry


In [155]:
# # If `dr_trips` is a GeoDataFrame and contains 'geometry' column
# if isinstance(dr_trips, gpd.GeoDataFrame) and 'StartGeometry' in dr_trips:
#     dr_trips['StartGeometry'] = dr_trips['StartGeometry'].apply(lambda geom: geom.__geo_interface__)

# # If `dr_trips` is a GeoDataFrame and contains 'geometry' column
# if isinstance(dr_trips, gpd.GeoDataFrame) and 'EndGeometry' in dr_trips:
#     dr_trips['EndGeometry'] = dr_trips['EndGeometry'].apply(lambda geom: geom.__geo_interface__)


In [156]:
# Create a KeplerGl map
#map_ = KeplerGl(height=600)

# Add the dr_trips DataFrame to the map
# Since the geometries are in WKT format, they should be automatically recognized and plotted
#map_.add_data(data=dr_trips, name='Trips')

# Display the map
#map_

For the Kmeans we agnowledge there is a problem with correlated features and that using PCA and GMM would solve this problem. However, we will use the Kmeans algorithm to cluster the data and then use the GMM to cluster the data. We will then compare the results of the two algorithms.

#### Explore the accessibility map

color code the map based on the differences between the pub transport and the time it takes by bike 
for all trips that start in 1 hub we get the difference between the time it takes by bike and the time it takes by public transport. We then average the differences for all trips that start in the same hub. We then color code the map based on the average differences.
Show a histogram of the differences 
